In [0]:
# Clone git repo
!git clone -b dataset https://github.com/HYUristic/DynaMotion

Cloning into 'DynaMotion'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 65 (delta 26), reused 54 (delta 18), pack-reused 0
Unpacking objects: 100% (65/65), done.


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.nn.init
import torchvision.transforms as transforms
import torchvision.datasets as dsets

class lstm_Model(nn.Module):
  def __init__(self, n_of_frames,n_of_cnn_frames, kernel_size, output_channel1, output_channel2):
    super(lstm_Model, self).__init__()

    self.n_of_frames = n_of_frames
    self.n_of_cnn_frames = n_of_cnn_frames
    self.kernel_size = kernel_size
    self.output_channel1 = output_channel1
    self.output_channel2 = output_channel2
    self.Rw1 = n_of_cnn_frames - kernel_size + 1
    self.Rh1 = 128 - kernel_size + 1
    self.Rw2 = (self.Rw1//2) - kernel_size + 1
    self.Rh2 = (self.Rh1//2) - kernel_size + 1
    self.Rw = self.Rw2//2
    self.Rh = self.Rh2//2
    self.sigmoid = nn.Sigmoid()

    self.layer1 = nn.Sequential(
        nn.Conv2d(1, output_channel1, kernel_size),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )

    self.layer2 = nn.Sequential(
        nn.Conv2d(output_channel1, output_channel2, kernel_size),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )

    self.layer3 = nn.LSTM(1*self.Rh*self.Rw, 128)

  def forward(self, x):
      out = self.layer1(x)
      out = self.layer2(out)
      out, (_, _) = self.layer3(out.view(x.size(0), 1, -1))
      out = self.sigmoid(out)

      return out

  def loss(self, output, target):
    loss = nn.MSELoss(output.view(1, -1), target)

    return loss

In [81]:
model = lstm_Model(12, 128, 5, 1, 1)
testdata = torch.zeros((1, 1,128, 128), dtype=torch.float64)
outputs= model(testdata.float())
outputs

tensor([[[0.4960, 0.5026, 0.4970, 0.5024, 0.5067, 0.4991, 0.4997, 0.4975,
          0.5043, 0.4974, 0.4997, 0.5022, 0.4953, 0.4974, 0.5064, 0.4977,
          0.4965, 0.4994, 0.5032, 0.4914, 0.4928, 0.4968, 0.5068, 0.5001,
          0.4980, 0.5040, 0.5080, 0.4998, 0.5064, 0.5061, 0.5032, 0.5056,
          0.4989, 0.4987, 0.5104, 0.4972, 0.5013, 0.4961, 0.4927, 0.4963,
          0.5025, 0.4965, 0.5032, 0.5007, 0.4984, 0.5043, 0.4985, 0.5090,
          0.4949, 0.4926, 0.5064, 0.4935, 0.5066, 0.4929, 0.5006, 0.5028,
          0.4995, 0.5024, 0.5009, 0.4994, 0.4958, 0.4913, 0.5034, 0.5049,
          0.5081, 0.5007, 0.5028, 0.5009, 0.5045, 0.4988, 0.5021, 0.4975,
          0.5041, 0.4995, 0.4972, 0.5042, 0.4995, 0.4942, 0.5002, 0.5013,
          0.4980, 0.5001, 0.5020, 0.5082, 0.5060, 0.5021, 0.4946, 0.5037,
          0.5043, 0.5061, 0.4939, 0.4985, 0.4921, 0.5003, 0.5001, 0.5005,
          0.4981, 0.4983, 0.4918, 0.5026, 0.4943, 0.5032, 0.5059, 0.4975,
          0.5014, 0.5007, 0.4987, 0.50

1